In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
h_waste = pd.read_csv("../data/Hazardous WasteIGD_EF_RCRA.csv")
toxic_release = pd.read_csv("../data/Toxic ReleasesIGD_EF_TRI.csv")
tn_pop = pd.read_csv("../data/tn_county_pop.csv")
swm_permits = pd.read_csv("../data/SWM Permits.csv")
rare_s = pd.read_csv("../data/Rare Species by County.csv", encoding='windows-1252')
streams = pd.read_csv("../data/Exceptional TN Streams.csv", encoding='windows-1252')

In [11]:
streams.head(5)

,HUC,Watershed Name,Waterbody,County,Description,Basis for Inclusion,From Lat,To Lat,From Long,To Long,Inclusion Date,Revision Date
0,5130104,Cumberland-South Fork,Gum Branch,Fentress,Portion in Big South Fork National River and R...,Big South Fork National River and Recreation A...,36.49606,36.48993,-84.74629,-84.75194,NaN,NaN
1,5130104,Cumberland-South Fork,Good Spring Branch,Scott,Portion in Big South Fork National River and R...,Big South Fork National River and Recreation Area,36.36300,36.36310,-84.73300,-84.73760,NaN,NaN
2,5130104,Cumberland-South Fork,Grand Branch,Scott,Portion in Big South Fork National River and R...,Big South Fork National River and Recreation Area,36.53670,36.54390,-84.71290,-84.72180,NaN,NaN
3,5130104,Cumberland-South Fork,Grassy Fork,Scott,Portion in Big South Fork National River and R...,Big South Fork National River and Recreation Area,36.57840,36.54390,-84.60480,-84.62980,NaN,12/5/2022
4,5130104,Cumberland-South Fork,Greasy Creek,Scott,Portion in Big South Fork NRRA including tribu...,Big South Fork NRRA,36.43150,36.43400,-84.59670,-84.59060,NaN,12/5/2022


In [18]:
streams['Basis for Inclusion'].fillna(False, inplace=True)

In [20]:
streams[streams['Basis for Inclusion'].str.lower().str.contains('endangered')].shape()

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [3]:
tn_waste = h_waste[h_waste['state_code']=='TN']
tn_toxic = toxic_release[toxic_release['state_code']=='TN']
tn_waste_center = swm_permits[(swm_permits['Permit Type']=='CONVENIENCE')&(swm_permits['Status']=='Active')]

In [ ]:
tn_toxic["county"] = tn_toxic["county_name"].str.split().str[0]
tn_waste.rename(columns={'county_name': 'county'}, inplace=True)
tn_pop.rename(columns={'county': 'county_name'}, inplace=True)
tn_pop["county"] = tn_pop["county_name"].str.split().str[0]
rare_s.rename(columns={'County': 'county'}, inplace=True)

In [ ]:
tw = tn_waste.groupby('county').agg({'registry_id': 'size', 'city_name': pd.Series.nunique}).reset_index().sort_values('city_name', ascending=False)
tt = tn_toxic.groupby('county').agg({'registry_id': 'size', 'city_name': pd.Series.nunique}).reset_index().sort_values('city_name', ascending=False)
tp = tn_pop[['county' , 'pop2024' , 'landArea' , 'density']]
tcc = tn_waste_center.groupby('County').agg({'Permit Number': 'count', 'City': pd.Series.nunique}).reset_index().sort_values('City', ascending=False)
rare = rare_s.groupby('county').agg({'Type': 'size', 'Category': pd.Series.nunique}).reset_index().sort_values('Category', ascending=False)
tcc.rename(columns={'County': 'county'}, inplace=True)
tcc['county'] = tcc['county'].str.upper()
tp['county'] = tp['county'].str.upper()
rare['county'] = rare['county'].str.upper()
tw.rename(columns={'registry_id': 'private_waste_loc_count', 'city_name': 'private_waste_city_count'}, inplace=True)
tt.rename(columns={'registry_id': 'toxic_release_loc_count', 'city_name': 'toxic_release_city_count'}, inplace=True)
tcc.rename(columns={'Permit Number': 'tn_convenience_loc_count', 'City': 'tn_convenience_city_count'}, inplace=True)
rare.rename(columns={'Type': 'tn_rare_count', 'Category': 'tn_rare_category'}, inplace=True)

In [6]:
cc_pop = pd.merge(tp, tcc, on='county', how='left')
cc_tt = pd.merge(cc_pop, tt, on='county', how='left')
cc_tt_tw = pd.merge(cc_tt, tw, on='county', how='left')
merged = pd.merge(cc_tt_tw, rare, on='county', how='left')
merged['conveniece_land_mile_ratio'] = merged['landArea'] / merged['tn_convenience_loc_count']
merged['conveniece_pop_ratio'] =  merged['pop2024'] / merged['tn_convenience_loc_count']
#merged.to_csv('tn_toxic',index=False)

In [7]:
merged

,county,pop2024,landArea,density,tn_convenience_loc_count,tn_convenience_city_count,toxic_release_loc_count,toxic_release_city_count,private_waste_loc_count,private_waste_city_count,tn_rare_count,tn_rare_category,conveniece_land_mile_ratio,conveniece_pop_ratio
0,SHELBY,903727,761,1187.551905,1.0,1.0,215.0,7.0,2669.0,32.0,26.0,8.0,761.000000,903727.000000
1,DAVIDSON,717317,503,1426.077535,5.0,1.0,140.0,8.0,2079.0,48.0,70.0,11.0,100.600000,143463.400000
2,KNOX,505958,508,995.980315,7.0,1.0,70.0,5.0,981.0,17.0,51.0,10.0,72.571429,72279.714286
3,HAMILTON,385126,542,710.564576,6.0,3.0,130.0,6.0,1164.0,31.0,67.0,11.0,90.333333,64187.666667
4,RUTHERFORD,373520,619,603.424879,15.0,4.0,60.0,4.0,380.0,15.0,84.0,11.0,41.266667,24901.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,HANCOCK,7066,222,31.828829,1.0,1.0,2.0,1.0,6.0,1.0,45.0,8.0,222.000000,7066.000000
91,MOORE,6780,129,52.558140,1.0,0.0,2.0,1.0,2.0,1.0,19.0,6.0,129.000000,6780.000000
92,VAN,6556,273,24.014652,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
93,LAKE,6194,166,37.313253,1.0,1.0,1.0,1.0,8.0,2.0,19.0,5.0,166.000000,6194.000000


In [8]:
#test=tn_toxic['county'].unique()

In [9]:
#county = tn_toxic['county_name'].str.split().str[0]